In [1]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
# from Src.config import hf_access_token, hf_model_cache_dir # noqa: E402
# os.environ["HF_HOME"] = hf_model_cache_dir 
# import torch

%load_ext autoreload
%autoreload 2

Load the model "gpt2" or "EleutherAI/pythia-6.9b"

In [3]:
from model import ModelFactory
model = ModelFactory.create("gpt2")

/opt/homebrew/anaconda3/envs/fact/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Cuda is not available, switching to cpu 
Loaded pretrained model gpt2 into HookedTransformer


Then load the dataset (Warning, select the right dataset for the model you loaded). Load also the ablator class


In [4]:
from dataset import BaseDataset
from experiment import Ablator


dataset = BaseDataset(path = "../data/full_data_sampled_gpt2.json",
                      model = model,
                      experiment="copyVSfact",
                      no_subject=True)
ablator = Ablator(model=model, dataset=dataset, experiment="copyVSfact", batch_size=20)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 No subject found in the dataset , proceeding with no subject data


Tokenizing and computing lengths:   0%|          | 0/10000 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tokenizing and computing lengths: 100%|██████████| 10000/10000 [00:14<00:00, 697.72it/s]


Set the heads that you want to modify, the value to multiply the heads and the token position that you want to modify (all, or attribute):

In [7]:
ablator.set_heads(heads=[(10,7), (11,10)], value=5, position="attribute")


Run the attention modification:


- mem --> logit of the factual token


- cp --> logit of the counterfactual token

- mem win --> number of factual predictions

- cp win --> number of counterfactual predictions

In [8]:
ablator.run()

Ablating: 100%|██████████| 24/24 [01:11<00:00,  2.96s/it]


,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,13.857364,14.130378,-0.273015,1.666047,2.702236,2.729101,5014.0,4614.0


In [8]:
# grid search for alpha/multiplier
result = []
for alpha in [2,5,10,100]:
    ablator.set_heads(heads=[(10,7), (11,10)], value=alpha, position="attribute")
    result.append(ablator.run())

Ablating:   0%|          | 0/24 [00:00<?, ?it/s]

Ablating: 100%|██████████| 24/24 [00:35<00:00,  1.47s/it]


In [9]:
result

[         mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.947168  15.336194 -1.389027  1.637497  2.689528  2.649626   3217.0   
 
    cp_win  
 0  6554.0  ,
          mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.857364  14.130377 -0.273015  1.666047  2.702235  2.729101   5014.0   
 
    cp_win  
 0  4614.0  ,
          mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.839895  13.989985 -0.150091  1.663305  2.688327  2.742588   5156.0   
 
    cp_win  
 0  4456.0  ,
          mem         cp      diff   mem_std    cp_std  diff_std  mem_win  \
 0  13.831625  13.926047 -0.094424  1.660842  2.670364  2.741903   5229.0   
 
    cp_win  
 0  4374.0  ]

## Use your own modification

You can pass to the model your own modification function. You can pass a list of hook (string, function) to the model using set_hooks method. The hooks should be consistent with the hook of the transformer lens library (https://neelnanda-io.github.io/TransformerLens/index.html)